In [11]:
import iso8601 #for date conversion
from datetime import datetime
import pandas as pd
import matplotlib
from xgboost import XGBClassifier
# matplotlib.use('nbagg')
import matplotlib.pyplot as plt
# plt.rcParams.update({'figure.max_open_warning': 0})
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from imblearn.over_sampling import SMOTE
from collections import Counter
from scipy.sparse import hstack,vstack
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.cross_validation import StratifiedKFold 
from collections import Counter, defaultdict
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import math
from sklearn.metrics import normalized_mutual_info_score
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings("ignore")

from mlxtend.classifier import StackingClassifier

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from importlib import reload
import gc



In [12]:
path = "mnt/ssd/kaggle-talkingdata2/competition_files/"
train_df = pd.read_csv(path+"train.csv")

In [13]:
train_df_class0 = train_df[train_df['is_attributed']==0]
train_df_class1 = train_df[train_df['is_attributed']==1]
chosen_indexes = np.random.choice(184447044,replace=False,size=456846)
train_df_class0 = train_df_class0.iloc[chosen_indexes]


In [15]:
frames = [train_df_class0,train_df_class1]
train_df = pd.concat(frames)
train_df = train_df.sort_values('click_time', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [16]:
train_df['click_time'] = pd.to_datetime(train_df['click_time'])
train_df['attributed_time'] = pd.to_datetime(train_df['attributed_time'])
train_df['hour'] = pd.to_datetime(train_df.click_time).dt.hour.astype('uint8')
train_df['day'] = pd.to_datetime(train_df.click_time).dt.day.astype('uint8')
train_df['dow'] = pd.to_datetime(train_df.click_time).dt.dayofweek.astype('uint8')
train_df['doy'] = pd.to_datetime(train_df.click_time).dt.dayofyear.astype('uint8')
gc.collect()

89

In [18]:
train_df.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,hour,day,dow,doy
103,204158,35,1,13,21,2017-11-06 15:41:07,2017-11-07 08:17:19,1,15,6,0,54
141,191880,3,1,18,379,2017-11-06 15:42:20,NaT,0,15,6,0,54
949,8286,25,1,19,259,2017-11-06 16:00:01,NaT,0,16,6,0,54
938,29531,21,1,19,128,2017-11-06 16:00:01,NaT,0,16,6,0,54
1798,64516,35,1,13,21,2017-11-06 16:00:02,2017-11-06 23:40:50,1,16,6,0,54


In [19]:
label = train_df['is_attributed'].values

In [20]:
train_df = train_df.drop(['click_time' ,'attributed_time' ],axis = 1)

In [21]:
train_df.head()

,ip,app,device,os,channel,is_attributed,hour,day,dow,doy
103,204158,35,1,13,21,1,15,6,0,54
141,191880,3,1,18,379,0,15,6,0,54
949,8286,25,1,19,259,0,16,6,0,54
938,29531,21,1,19,128,0,16,6,0,54
1798,64516,35,1,13,21,1,16,6,0,54


In [23]:
# split the data into test and train by maintaining same distribution of output varaible 'y_true' [stratify=y_true]
X_train, test_df, y_train, y_test = train_test_split(train_df, label, shuffle =False , test_size=0.2)
# split the train data into train and cross validation by maintaining same distribution of output varaible 'y_train' [stratify=y_train]
train_df, cv_df, y_train, y_cv = train_test_split(X_train, y_train, shuffle =False , test_size=0.2)